In [1]:
import pandas as pd
import os
import torch
from model.database_util import *

In [2]:
import torch
from model.util import Normalizer

# cost_norm = Normalizer(1, 100)
# cost_norm = Normalizer(-3.61192, 12.290855)
#cost_norm = Normalizer(5, 2611)
cost_norm = Normalizer(8.26, 11.12)

## Define Model

In [3]:
class Args:
    # bs = 1024
    # SQ: smaller batch size
    bs = 64
    # bs = 10
    #lr = 0.001
    lr = 0.001
    # epochs = 200
    epochs = 50
    clip_size = 50
    embed_size = 64
    pred_hid = 128
    ffn_dim = 128
    head_size = 12
    n_layers = 8
    dropout = 0.1
    sch_decay = 0.6
    # device = 'cuda:0'
    device = 'cpu'
    newpath = 'job_queries_training'
    to_predict = 'cost'
args = Args()

import os
if not os.path.exists(args.newpath):
    os.makedirs(args.newpath)

In [4]:
from model.model import QueryFormer

model = QueryFormer(emb_size = args.embed_size ,ffn_dim = args.ffn_dim, head_size = args.head_size, \
                 dropout = args.dropout, n_layers = args.n_layers, \
                 use_sample = False, use_hist = False, \
                 pred_hid = args.pred_hid
                )

In [5]:
from model.dataset import PlanTreeDataset

## TEST - Loading 10 tensors

In [6]:
import os
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from model.dataset import PlanTreeDataset  # Assuming PlanTreeDataset is defined elsewhere
import json

# Path to the tensors folder
tensors_dir = "./job_queries/tensors"

# Validate if the tensors directory exists
if not os.path.exists(tensors_dir):
    raise FileNotFoundError(f"Tensors directory '{tensors_dir}' not found.")

# Get all tensor file paths from the directory
tensor_files = sorted(os.listdir(tensors_dir))

# Validate if tensor files exist in the directory
if not tensor_files:
    raise FileNotFoundError(f"No tensor files found in '{tensors_dir}'.")

# Initialize lists to store tensor components
x_list = []
rel_pos_list = []
attn_bias_list = []
heights_list = []
cost_labels_list = []
raw_costs_list = []

# Load tensors dynamically
for tensor_file in tensor_files:
    tensor_path = os.path.join(tensors_dir, tensor_file)
    try:
        loaded_tensors = torch.load(tensor_path)
        # Append components to respective lists
        x_list.append(loaded_tensors["x"])
        rel_pos_list.append(loaded_tensors["rel_pos"])
        attn_bias_list.append(loaded_tensors["attn_bias"])
        heights_list.append(loaded_tensors["heights"])
        cost_labels_list.append(loaded_tensors["cost_labels"])
        raw_costs_list.append(loaded_tensors["raw_costs"])
    except Exception as e:
        print(f"Error loading tensor file '{tensor_file}': {e}")

# Generate indices for splitting
num_examples = len(x_list)
if num_examples == 0:
    raise ValueError("No valid tensors loaded for dataset creation.")
    
all_indices = np.arange(num_examples)

# Perform train-validation split with fixed seed for reproducibility
train_indices, val_indices = train_test_split(all_indices, test_size=0.2, random_state=0)

# Create training and validation datasets
train_dataset = PlanTreeDataset(
    len(train_indices),
    [x_list[i] for i in train_indices],
    [attn_bias_list[i] for i in train_indices],
    [rel_pos_list[i] for i in train_indices],
    [heights_list[i] for i in train_indices],
    [cost_labels_list[i] for i in train_indices],
    [raw_costs_list[i] for i in train_indices]
)

val_dataset = PlanTreeDataset(
    len(val_indices),
    [x_list[i] for i in val_indices],
    [attn_bias_list[i] for i in val_indices],
    [rel_pos_list[i] for i in val_indices],
    [heights_list[i] for i in val_indices],
    [cost_labels_list[i] for i in val_indices],
    [raw_costs_list[i] for i in val_indices]
)

# Save validation indices and file names
val_file_names = [tensor_files[i] for i in val_indices]
val_data = {
    "val_indices": val_indices.tolist(),
    "file_names": val_file_names,
}

val_data_file = "./val_data.json"
with open(val_data_file, "w") as f:
    json.dump(val_data, f)

# Save training indices and file names
train_file_names = [tensor_files[i] for i in train_indices]
train_data = {
    "train_indices": train_indices.tolist(),
    "file_names": train_file_names,
}

train_data_file = "./train_data.json"
with open(train_data_file, "w") as f:
    json.dump(train_data, f)

# Print information
print("Training Dataset length:", len(train_dataset))
print("Validation Dataset length:", len(val_dataset))
print(f"Validation data saved to {val_data_file}")
print(f"Training data saved to {train_data_file}")

Training Dataset length: 1865
Validation Dataset length: 467
Validation data saved to ./val_data.json
Training data saved to ./train_data.json


In [7]:
import pandas as pd
import json

# File paths
val_data_file = "./val_data.json"
train_data_file = "./train_data.json"

# Load validation data from JSON file
with open(val_data_file, "r") as f:
    loaded_val_data = json.load(f)

loaded_val_indices = loaded_val_data["val_indices"]
loaded_val_file_names = loaded_val_data["file_names"]

# Create a DataFrame for validation data
val_data_df = pd.DataFrame({
    "val_index": loaded_val_indices,
    "filename": loaded_val_file_names
})

# Display the Validation DataFrame
print("Validation DataFrame:")
print(val_data_df.head())

# Save validation data to CSV
val_data_df.to_csv("./val_data.csv", index=False)
print("Validation data saved to ./val_data.csv")

# Load training data from JSON file
with open(train_data_file, "r") as f:
    loaded_train_data = json.load(f)

loaded_train_indices = loaded_train_data["train_indices"]
loaded_train_file_names = loaded_train_data["file_names"]

# Create a DataFrame for training data
train_data_df = pd.DataFrame({
    "train_index": loaded_train_indices,
    "filename": loaded_train_file_names
})

# Display the Training DataFrame
print("Training DataFrame:")
print(train_data_df.head())

# Save training data to CSV
train_data_df.to_csv("./train_data.csv", index=False)
print("Training data saved to ./train_data.csv")


Validation DataFrame:
   val_index                                  filename
0       1056  query_1952_2024-12-03-21.34.02.429279.pt
1       2323   query_993_2024-12-03-21.10.08.618148.pt
2        655  query_1591_2024-12-03-21.23.17.861948.pt
3       2101   query_793_2024-12-03-21.05.49.462784.pt
4       1401  query_2262_2024-12-03-21.44.00.539251.pt
Validation data saved to ./val_data.csv
Training DataFrame:
   train_index                                  filename
0          812  query_1732_2024-12-03-21.27.45.652122.pt
1          233  query_1210_2024-12-03-21.14.50.599373.pt
2         1380  query_2243_2024-12-03-21.43.30.935782.pt
3         1752   query_479_2024-12-03-20.59.07.992844.pt
4         1334  query_2201_2024-12-03-21.42.07.400700.pt
Training data saved to ./train_data.csv


In [8]:
len(train_dataset)

1865

In [9]:
len(train_dataset[:10])

10

In [10]:
# Example numpy label
import numpy as np
import torch.nn as nn
import importlib

from model import trainer
importlib.reload(trainer)
from  model.trainer import train_single, train


crit = nn.MSELoss()

# Train the model with the numpy label
# trained_model = train_single(model, dataset, dataset, crit, cost_norm, args)
model, best_model_path, train_embeddings, val_embeddings = train(model, train_dataset, val_dataset, crit, cost_norm, args)


running epoch: 0
final.shape: torch.Size([1920, 256])


final.shape: torch.Size([1920, 256])


final.shape: torch.Size([1920, 256])


final.shape: torch.Size([1920, 256])


final.shape: torch.Size([1920, 256])


...

final.shape: torch.Size([1920, 256])
final.shape: torch.Size([1920, 256])


final.shape: torch.Size([1920, 256])
final.shape: torch.Size([1920, 256])


final.shape: torch.Size([1920, 256])
final.shape: torch.Size([1920, 256])


final.shape: torch.Size([570, 256])
ps: [39100.617  15718.277  39100.617  11286.7705 67027.53   13871.979
 67027.53   59201.027  39100.582  11286.759  67027.53   11286.759
 59200.97   59200.97   67027.53   59200.97   39100.617  67027.53
 59200.97   24885.621  59200.97   39100.617  16713.79   59201.027
 67027.53   67027.53   11286.759  67027.53   15718.293  67027.53
 59201.027  59200.97   39100.617  59200.97   67027.53   17467.324
 67027.53   39100.617  11286.759  59200.97   59201.027  67027.53
 67027.53   59200.97   59201.027  39100.582  67027.53   39100.617
 59201.027  14870.7    11286.7705 11286.759  13871.979  59200.97
 39100.617  39100.617  59201.027  67027.53   67027.53   67027.53
 67027.53   11286.7705 39100.582  11286.7705 59200.97   67027.53
 59200.97   67027.53   11286.759  11286.7705 67027.53   67027.53
 39100.617  11286.759  39100.617  59201.027  11286.759  67027.53
 13872.006  67027.53   67027.53   13872.006  11286.7705 13871.979
 11286.759  14870.7    59200.97   67027.53  

ls: [tensor([35153.], dtype=torch.float64), tensor([9951.], dtype=torch.float64), tensor([38815.], dtype=torch.float64), tensor([14175.], dtype=torch.float64), tensor([66821.], dtype=torch.float64), tensor([9951.], dtype=torch.float64), tensor([66821.], dtype=torch.float64), tensor([58513.], dtype=torch.float64), tensor([39421.], dtype=torch.float64), tensor([9951.], dtype=torch.float64), tensor([66821.], dtype=torch.float64), tensor([14187.], dtype=torch.float64), tensor([58512.], dtype=torch.float64), tensor([58512.], dtype=torch.float64), tensor([66822.], dtype=torch.float64), tensor([58512.], dtype=torch.float64), tensor([35343.], dtype=torch.float64), tensor([66823.], dtype=torch.float64), tensor([58512.], dtype=torch.float64), tensor([19798.], dtype=torch.float64), tensor([58512.], dtype=torch.float64), tensor([38590.], dtype=torch.float64), tensor([9951.], dtype=torch.float64), tensor([58513.], dtype=torch.float64), tensor([66821.], dtype=torch.float64), tensor([66821.], dtype=t

In [11]:
best_model_path

'-4892345993358029136.pt'

In [12]:
len(loaded_val_indices[:10])

10

In [13]:
len(val_embeddings)

467

In [14]:
import pandas as pd
import numpy as np

# Assume these are loaded or computed
# val_embeddings: List or numpy array of embeddings (e.g., shape [num_examples, embedding_dim])
# loaded_val_indices: List of validation indices
# loaded_file_names: List of validation file names

# Convert val_embeddings to a numpy array if it's not already
val_embeddings = np.array(val_embeddings)

# Ensure the size of all inputs matches
if len(loaded_val_indices) != len(loaded_val_file_names) or len(loaded_val_indices) != len(val_embeddings):
    raise ValueError("The sizes of validation indices, filenames, and embeddings must match.")

# Create a DataFrame
val_data_df = pd.DataFrame({
    "val_index": loaded_val_indices,
    "filename": loaded_val_file_names,
    "embedding": list(val_embeddings)  # Store embeddings as a list of numpy arrays
})

# Save the DataFrame to a CSV file
csv_file_path = "validation_embeddings.csv"
val_data_df.to_csv(csv_file_path, index=False)

# Optionally save embeddings in a separate binary file for efficient storage
embeddings_file_path = "validation_embeddings.npy"
np.save(embeddings_file_path, val_embeddings)

print(f"Validation DataFrame created and saved to {csv_file_path}")
print(f"Embeddings saved to {embeddings_file_path}")

# Print the first few rows for verification
print(val_data_df.head())


Validation DataFrame created and saved to validation_embeddings.csv
Embeddings saved to validation_embeddings.npy
   val_index                                  filename  \
0       1056  query_1952_2024-12-03-21.34.02.429279.pt   
1       2323   query_993_2024-12-03-21.10.08.618148.pt   
2        655  query_1591_2024-12-03-21.23.17.861948.pt   
3       2101   query_793_2024-12-03-21.05.49.462784.pt   
4       1401  query_2262_2024-12-03-21.44.00.539251.pt   

                                           embedding  
0  [-6.3221054, -1.2161089, 0.43031985, 13.114403...  
1  [-7.1674953, 1.2622585, -1.4078889, 11.233791,...  
2  [-6.3221154, -1.216113, 0.43032083, 13.114399,...  
3  [-7.363716, -0.7742083, 1.3781253, 10.9354, 2....  
4  [1.0355659, 6.7917485, -3.5958993, -5.911288, ...  


In [15]:
val_data_df.head()

,val_index,filename,embedding
0,1056,query_1952_2024-12-03-21.34.02.429279.pt,"[-6.3221054, -1.2161089, 0.43031985, 13.114403..."
1,2323,query_993_2024-12-03-21.10.08.618148.pt,"[-7.1674953, 1.2622585, -1.4078889, 11.233791,..."
2,655,query_1591_2024-12-03-21.23.17.861948.pt,"[-6.3221154, -1.216113, 0.43032083, 13.114399,..."
3,2101,query_793_2024-12-03-21.05.49.462784.pt,"[-7.363716, -0.7742083, 1.3781253, 10.9354, 2...."
4,1401,query_2262_2024-12-03-21.44.00.539251.pt,"[1.0355659, 6.7917485, -3.5958993, -5.911288, ..."


In [16]:
print(f"Number of training embeddings (best epoch): {len(train_embeddings)}")
print(f"First training embedding shape: {train_embeddings[0].shape}")
print(f"Number of validation embeddings (best epoch): {len(val_embeddings)}")
print(f"First validation embedding shape: {val_embeddings[0].shape}")


Number of training embeddings (best epoch): 1865
First training embedding shape: (521,)
Number of validation embeddings (best epoch): 467
First validation embedding shape: (521,)


In [17]:
print(f"First validation embedding shape: {val_embeddings[0]}")

First validation embedding shape: [ -6.3221054   -1.2161089    0.43031985  13.114403     0.597692
   5.5710177   -1.4463073   -7.6218276    6.958821   -14.633663
  -3.2873127   -0.4089021    6.029482   -27.001808     1.9260159
  -1.0277863   13.548631   -15.065728    -8.411574   -30.127523
  -7.6771083    1.8954351   14.028365     5.2963824   -6.392975
 -12.7531805   -6.0769215   13.20106      3.7452826  -13.305846
   6.6460714   -0.9090615    1.6611434    3.2162087   -0.80632
  -5.912134     0.34829015 -15.1260395   -0.19040892  13.345829
  -9.295478   -16.171478     0.4497921   -2.315885   -12.691885
   1.805241    31.083757   -10.805419   -13.88535      3.5872593
  -3.3870864    1.8303446  -12.171584     2.2706897    3.740867
  28.458776    18.14137     13.3015     -11.954642     5.277808
 -11.089407     5.0655594  -11.487695     0.37082273  -9.386708
   9.785867     2.0261111    3.7672932   -6.670656   -12.673987
  32.099094   -21.028584    13.668743    10.898011     9.106477
  -7.

In [18]:
type(val_embeddings[0])

numpy.ndarray

In [19]:
print(f"First validation embedding shape: {val_embeddings[1]}")

First validation embedding shape: [-7.1674953e+00  1.2622585e+00 -1.4078889e+00  1.1233791e+01
  3.4764476e+00  5.3047266e+00 -3.2085449e-01 -1.9125725e+00
  1.5687717e+00 -1.2804629e+01 -2.7944949e+00 -1.4415908e+00
  6.7057371e+00 -2.9670046e+01  2.3629963e+00 -4.7606177e+00
  1.1895366e+01 -1.0634305e+01 -6.1758637e+00 -3.1145290e+01
 -9.8893299e+00  3.4185634e+00  1.3296408e+01  5.6195412e+00
 -6.3199635e+00 -1.2277816e+01 -7.4814234e+00  1.5109468e+01
  4.4720697e+00 -1.3546993e+01  5.2070990e+00 -1.9114726e+00
  1.7673786e+00  4.2464638e+00 -1.1493626e+00 -2.4278011e+00
 -8.9926535e-01 -8.9888725e+00 -1.4308406e+00  1.1384814e+01
 -7.4287424e+00 -1.7852072e+01 -2.8457353e+00 -2.9268486e+00
 -1.6674889e+01  5.0513759e+00  3.1327429e+01 -9.7184973e+00
 -1.2802489e+01  4.3773937e+00 -4.6619678e+00  1.6481143e+00
 -8.0541363e+00  4.8522978e+00 -3.3938062e-01  3.0559837e+01
  1.8901333e+01  1.4126292e+01 -9.7827978e+00  4.6072540e+00
 -1.2705544e+01  3.5059650e+00 -9.2408590e+00  2.58

In [20]:
len(val_embeddings[0])

521

In [21]:
# import numpy as np

# # Disable truncation
# np.set_printoptions(threshold=np.inf, linewidth=1000)

# for i, emb in enumerate(val_embeddings):
#     print(f"Full Embedding {i}:\n{emb}")


In [22]:
import json
import numpy as np

# Ensure embeddings are numpy arrays
train_embeddings = np.array(train_embeddings)
val_embeddings = np.array(val_embeddings)

# Paths for saving the JSON files
train_embeddings_file = "./train_embeddings.json"
val_embeddings_file = "./val_embeddings.json"

# Create dictionaries with file names as keys and embeddings as values
train_embedding_dict = {train_file_name: embedding.tolist() for train_file_name, embedding in zip(loaded_train_file_names, train_embeddings)}
val_embedding_dict = {val_file_name: embedding.tolist() for val_file_name, embedding in zip(loaded_val_file_names, val_embeddings)}

# Save training embeddings to JSON
with open(train_embeddings_file, "w") as f:
    json.dump(train_embedding_dict, f)

# Save validation embeddings to JSON
with open(val_embeddings_file, "w") as f:
    json.dump(val_embedding_dict, f)

print(f"Train embeddings saved to {train_embeddings_file}")
print(f"Validation embeddings saved to {val_embeddings_file}")

Train embeddings saved to ./train_embeddings.json
Validation embeddings saved to ./val_embeddings.json


In [23]:
import pandas as pd
import json

# Path to the JSON file
val_embeddings_file = "./val_embeddings.json"  # Replace with train_embeddings.json for training data

# Load the JSON file
with open(val_embeddings_file, "r") as f:
    embeddings_data = json.load(f)

# Convert JSON data to a DataFrame
val_embeddings_df = pd.DataFrame({
    "file_name": list(embeddings_data.keys()),
    "embedding": list(embeddings_data.values())
})

# Display the DataFrame
print(val_embeddings_df.head())

# Optionally save it to a CSV
val_embeddings_df.to_csv("val_embeddings.csv", index=False)
print("Validation embeddings saved to val_embeddings.csv")


                                  file_name  \
0  query_1952_2024-12-03-21.34.02.429279.pt   
1   query_993_2024-12-03-21.10.08.618148.pt   
2  query_1591_2024-12-03-21.23.17.861948.pt   
3   query_793_2024-12-03-21.05.49.462784.pt   
4  query_2262_2024-12-03-21.44.00.539251.pt   

                                           embedding  
0  [-6.322105407714844, -1.2161089181900024, 0.43...  
1  [-7.167495250701904, 1.262258529663086, -1.407...  
2  [-6.322115421295166, -1.2161129713058472, 0.43...  
3  [-7.363716125488281, -0.7742083072662354, 1.37...  
4  [1.0355658531188965, 6.791748523712158, -3.595...  


Validation embeddings saved to val_embeddings.csv
